In [ ]:
import pandas_gbq
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

chamado_1746 = pd.read_parquet("../data/chamado_1746.parquet")
bairro = pd.read_parquet("../data/bairro.parquet")
ocupacao_eventos = pd.read_parquet("../data/rede_hoteleira_ocupacao_eventos.parquet")

## Análise exploratória de dados inicial

In [ ]:
bairro.head()

In [ ]:
ocupacao_eventos.head()

In [ ]:
chamado_1746.sample(5)

In [ ]:
chamado_1746.info()

In [ ]:
chamado_1746.describe().T

In [ ]:
chamado_1746.describe(include="object").T

In [ ]:
columns = [
    "prazo_unidade",
    "dentro_prazo",
    "status",
    "situacao",
    "tipo_situacao",
    "justificativa_status",
    "unidade_organizacional_ouvidoria",
    "prazo_tipo",
]

# Prazo tipo: D (diagnostico), F (finalização)
# Prazo unidade: D (dias), H (horas)

fig, axes = plt.subplots(3, 3, figsize=(20, 20))
fig.suptitle("Distribuição de variáveis categóricas")
for i, col in enumerate(columns):
    ax = axes[i // 3, i % 3]
    sns.countplot(x=col, data=chamado_1746, ax=ax)
    ax.set_title(col)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.tight_layout()
plt.show()

- A maioria dos chamados nos anos de 2022 e 2023 tiveram prazo definido em dias, sendo que a maioria foi resolvido dentro do prazo. 
- "Fechado com solução" é o status mais presente nos chamados, indicando que a maioria dos chamados foi resolvida. 
- A situação Encerrada é a mais presente nos chamados, indicando que a maioria dos chamados foi encerrada.
- Os chamados, em sua maioria, não foram feitos via ouvidoria.
- Os chamados que possuem campo de justiticativa preenchido tinham como valor "Não há previsão para execução do serviço no local" em sua maioria.

## Exploração dos meses

Visualização dos dados de cada mês



In [ ]:
meses_nomes = [
    "Janeiro",
    "Fevereiro",
    "Março",
    "Abril",
    "Maio",
    "Junho",
    "Julho",
    "Agosto",
    "Setembro",
    "Outubro",
    "Novembro",
    "Dezembro",
]

chamado_1746["mes"] = chamado_1746["data_inicio"].dt.month
chamado_1746["mes"] = chamado_1746["mes"].apply(lambda x: meses_nomes[x - 1])
chamado_1746["mes"] = pd.Categorical(
    chamado_1746["mes"], categories=meses_nomes, ordered=True
)

fig, ax = plt.subplots(figsize=(15, 8))
sns.countplot(x="mes", data=chamado_1746, hue="tipo_situacao", ax=ax)
ax.set_title("Distribuição do tipo de situação por mês")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.countplot(x="mes", data=chamado_1746, hue="justificativa_status", ax=ax)
ax.set_title("Distribuição da justificativa de status por mês")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.countplot(x="mes", data=chamado_1746, hue="dentro_prazo", ax=ax)
ax.set_title("Distribuição de chamadas dentro e fora do prazo mensalmente")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.countplot(x="mes", data=chamado_1746, hue="situacao", ax=ax)
ax.set_title("Distribuição da situação por mês")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.countplot(x="mes", data=chamado_1746, ax=ax)
ax.set_title("Frequência de chamados por mês")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.countplot(
    x="mes", data=chamado_1746[chamado_1746["tipo_situacao"] == "Atendido"], ax=ax
)
ax.set_title("Frequência de chamados atendidos por mês")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.countplot(
    x="mes", data=chamado_1746[chamado_1746["tipo_situacao"] == "Não atendido"], ax=ax
)
ax.set_title("Frequência de chamados não atendidos por mês")

- Janeiro é o mês com mais chamados, seguido por março e fevereiro respectivamente.
- Os chamados em janeiro, maio e março foram os que foram mais atendidos.
- Chamados não atendidos foram mais frequentes em março, janeiro e fevereiro.
- Fevereiro foi o mês com mais chamados não encerrados.
- Fevereiro também foi o mês com mais chamados fora do prazo.

## Exploração por tipos

In [ ]:
top5 = chamado_1746["tipo"].value_counts().head(5)
top5.plot(kind="bar", color="blue")
plt.title("Top 5 tipos de chamados")

In [ ]:
chamado_1746["status"].value_counts()

In [ ]:
dentro_prazo = chamado_1746["dentro_prazo"]
df_dentro_prazo = chamado_1746[chamado_1746["dentro_prazo"] == "No prazo"]
print(df_dentro_prazo.shape[0])
tipos = df_dentro_prazo["tipo"].value_counts().head(5)
tipos.plot(kind="bar", color="blue")
plt.title("Top 5 tipos de chamados dentro do prazo")

In [ ]:
# top categorias/tipos fora do prazo
fora_prazo = chamado_1746["dentro_prazo"]
df_fora_prazo = chamado_1746[chamado_1746["dentro_prazo"] != "No prazo"]
print(df_fora_prazo.shape[0])
tipos = df_fora_prazo["tipo"].value_counts().head(5)
tipos.plot(kind="bar", color="blue")
plt.title("Top 5 tipos de chamados fora do prazo")

In [ ]:
# top categorias/tipos em andamento, mas fora do prazo
fora_prazo = chamado_1746["dentro_prazo"]
df_fora_prazo = chamado_1746[chamado_1746["dentro_prazo"] != "No prazo"]
df_em_andamento = df_fora_prazo[df_fora_prazo["data_fim"].isna() == True]
df_em_andamento = df_em_andamento[df_em_andamento["status"] == "Em Andamento"]
print(df_em_andamento.shape[0])
tipos = df_em_andamento["tipo"].value_counts().head(5)
tipos.plot(kind="bar", color="blue")
plt.title("Top 5 tipos de chamados em andamento, mas fora do prazo")

- Estacionamento irregular e Iluminação pública são os tipos de chamados mais frequentes.
- Em chamados dentro do prazo, "Estacionamento Irregular" é o tipo mais frequente.
- Em chamados fora do prazo, "Iluminação pública" é o tipo mais frequente.
- Dos chamados que estavam em andamento no período analisado, mas fora do prazo, "Manejo Árboreo" era o tipo mais frequente.